### Faster counting using sorting
This notebook demonstrates how sorting can be used to improve access locality. Here we use sorting in the context of a single CPU computation. The lessons are even more important in the context of a multi-CPU cluster, as will be shown in a separate notebook.

This notebook can be run without spark, using just Jupyter.

#### The task

We are given a text file, here we are using Moby Dick by Herman Melville which can be downloaded from [here](http://faculty.washington.edu/stepp/courses/2004autumn/tcss143/lectures/files/2004-11-08/mobydick.txt).

Our task is to read the file, separate it into words, make all words lower-case, and count the number of occurances of each word.

In [ ]:
# initialization for grading
import sys
import os 
testPath = '/'.join(os.getcwd().split('/')[:-1]) + "/Tester"
sys.path.insert(0, testPath )

pickleFile = testPath+ "/SimpleCount.pkl"

In [ ]:
from string import lower,strip
import re
%pylab inline

### Reading in the file
We read in the file, split it into words and create a list called `all` which contains all of the words in the document

In [ ]:
%%time
import urllib
data_dir='../../Data'
filename='Moby-Dick.txt'

f = urllib.urlretrieve("https://mas-dse-open.s3.amazonaws.com/"+filename, data_dir+'/'+filename)

# Check that the text file is where we expect it to be
!ls -l $data_dir/$filename

In [ ]:
#%%time
file=open(data_dir+'/'+filename,'r')

all=[]
for line in file.readlines():
    line=lower(strip(line))
    if len(line)==0:
        continue
    words=[w for w in re.split(r'\W+',line) if len(w)>0]
    #print line, words
    all+=words
print 'the book contains',len(all),'words'

### Simple sort
First, lets try counting words using the most straight-forward approach.
We create a dictionary `D` and then go through the list of words `all`. For each word we increment the corresponding entry in `D` if the word exists as a key in the dictionary, if it is not in the dictionary, we add it to the dictionary

In [ ]:
%%time
def simple_count(list):
    D={}
    for w in list:
        if w in D.keys():
            D[w]+=1
        else:
            D[w]=1
    return D
D=simple_count(all)

### Simple count is slow!
It takes, on my laptop, about a minute to finish the counting.

We check that the output makes sense by printing the 10 most common words.

In [ ]:
S=sorted(D.items(),key=lambda d:d[1],reverse=True)
S[:10]

### Sorted count
Next, we show a faster way to count. We first sort the words alphabetically. Then, when we iterate through the sorted list, all of the occurances of any word appear consecutively, allowing us to quickly count them using just one counter. This counter is added to the dictionary when this element of the list is different than the previous one.

In [ ]:
%%time
def sort_count(list):
    S=sorted(list)
    D={}
    current=''
    count=0
    for w in S:
        if current==w:
            count+=1
        else:
            if current!='':
                D[current]=count
            count=1
            current=w
    return D
D=sort_count(all)

In [ ]:
len(D)

### Discussion
As we see, `sort_count` is about 350 times faster than `simple_count`. 

**why?**  
The reason is that sorting the list of words improves the locality of memory access for the table `D`. 
`simple_count` accesses the elements in `D` in the order of the words in the text. In natural text you rarely see the same word appearing several times consecutively. As there are likely to be large gaps between occurances of the same word, when an element of `D` needs to be updated, it is unlikely to be in the cache. The result is a large number of cache misses, resulting in a long run time.

In contrast, after the words are sorted, all of the occurances of each word are consecutive. This means that sort_count makes exactly one write access to create each element of `D`, and will never require this element again.

Below is a printout of a segment of the original list and a printout of a segment of the sorted list. In fact, if you pick a random segment of the sorted list, it is likely to consist of repetitions of a single word.

In [ ]:
print '=== unsorted list:'
print ','.join(all[100214:100240])
S=sorted(all)
print '=== sorted list:'
print ','.join(S[100214:100240])

### Built in Counter
Another way to achieve good performance is to use `Counter` from the standard python library [collections](https://docs.python.org/2/library/collections.html)

We include this here for the sake of comparison

In [ ]:
from collections import Counter

### Performance scaling
We now run the three counting methods: `simple`, `sort` and `counter` on lists of different length and then plot the performances on a log-log scale.

Why do we use log-log graphs?
1. Because it makes visible the performance across several orders of magnitude.
2. Because if the relationship between the list length $n$ and the run time $t$ is of the form $t=n^\alpha$ then the relationship between $t$ and $n$ on the log-log scale appears linear, with a slope of $\alpha$.

In [ ]:
from time import time
l=len(all)
Ln=[];Lsimple=[];Lsort=[];Lcounter=[]
for i in range(10):
    time1=time()
    n=l/(2**i)
    simple_count(all[:n])
    time2=time()
    sort_count(all[:n])
    time3=time()
    Counter(all[:n])
    time4=time()
    Ln.append(n)
    t_simple=time2-time1; Lsimple.append(t_simple)
    t_sort=time3-time2; Lsort.append(t_sort)
    t_counter=time4-time3; Lcounter.append(t_counter)

    print 'len=%5d, simple=%f, sort=%f, counter=%f'%(n,t_simple,t_sort,t_counter)
 

In [ ]:
loglog(Ln,Lsimple,label='simple count');
loglog(Ln,Lsort,label='sort count');
loglog(Ln,Lcounter,label='counter')
grid();
title('Performance of simple count and sort count')
legend(loc='best')
xlabel('length of list')
ylabel('run time');

### Conclusions
We see from the graph that `sort` and `counter` have very similar run times, and are both better than `simple`. The gap between `sort` and `simple` increases as the length of the list increases. This makes sense because the longer the list, the higher the frequency of cache misses.

We also see that while for `sort` and `counter` the relationship is approximately linear $t \approx n$. The relation for `simple` is of the form $t \approx n^{1.5}$

## Exercise 1 

A `k`-mer is a sequence of `k` consecutive words. 

For example, the `3`-mers in the line `you are my sunshine my only sunshine` are

* `you are my`
* `are my sunshine`
* `my sunshine my`
* `sunshine my only`
* `my only sunsine`

For the sake of simplicity we consider only the `k`-mers that appear in a single line. In other words, we ignore `k`-mers that span more than one line.

Write a function **compute_kmers**, to return the list of `k`-mers in a given text for a given `k`.

######  <span style="color:blue">Code:</span>
```python
text = ['you are my sunshine my only sunshine']
compute_kmers(text,3)
```
######  <span style="color:magenta">Output:</span>
['you are my', 'are my sunshine', 'my sunshine my', 'sunshine my only', 'my only sunsine']

In [ ]:
def compute_kmers(text,k):
    kmers = []
    # your implementation goes here
    return kmers

In [ ]:
import SimpleCount
SimpleCount.exercise0_1(pickleFile, compute_kmers)

## Exercise 2

Given a list of k-mers, write a function **count_kmers**, to return the dictionary with key as `k`-mer and value as the number of times it has occurred (the count) in the input list.

######  <span style="color:blue">Code:</span>
```python
kmers = ['you are my', 'are my sunshine', 'my sunshine my', 'sunshine my only', 'my only sunshine']
count_kmers(kmers)
```
######  <span style="color:magenta">Output:</span>

{'you are my' : 1, 'are my sunshine' : 1, 'my sunshine my' : 1, 'sunshine my only' : 1, 'my only sunsine' : 1}

In [ ]:
def count_kmers(kmers):
    kmers_count = dict()
    # your implementation goes here 
    return kmers_count

In [ ]:
import SimpleCount
SimpleCount.exercise0_2(pickleFile, count_kmers)

## Exercise 3 

Given the dictionary of k-mer counts from exercise 2, write a function **sort_counts**, to sort the k-mers in descending order by its count. Return a list of tuples. 

* `Each tuple is of the form (kmer, count).`
* `If two k-mers have same count, then sort them lexicographically.`
    
######  <span style="color:blue">Code:</span>
```python
kmers_counts =  {'you are my' : 1, 'are my sunshine' : 1, 'my sunshine my' : 1, 'sunshine my only' : 1, 'my only sunsine' : 1}
sort_counts(kmers_counts)
```
######  <span style="color:magenta">Output:</span>
[('are my sunshine', 1) , ('my only sunsine' , 1) , ('my sunshine my', 1), ('sunshine my only', 1) , ('you are my', 1)]

In [ ]:
def sort_counts(kmers_counts):
    sorted_counts = []
    # your implementation goes here
    return sorted_counts

In [ ]:
import SimpleCount
SimpleCount.exercise0_3(pickleFile, sort_counts)

## Exercise 4 

Given a list of lines, Write a function, to return the list of tuples containing top `n` k-mers with its count from the given text for a given n, k.


######  <span style="color:blue">Code:</span>
```python
n=2
k=3
text = ['you are my sunshine my only sunsine']
get_top_n_kmers(text,n,k)
```
######  <span style="color:magenta">Output:</span>
    [('are my sunshine', 1) , ('my only sunsine' , 1)]

In [ ]:
def get_top_n_kmers(text,n,k):
    kmers = compute_kmers(text,k)
    kmers_count = count_kmers(kmers)
    sorted_counts = sort_counts(kmers_count)
    #SOLUTION BEGINS
    top_n_kmers = []
    #SOLUTION ENDS
    print 'most common %d-mers\n'%k,'\n'.join(['%d:\t%s'%c for c in top_n_kmers])
    return top_n_kmers

In [ ]:
import SimpleCount
SimpleCount.exercise0_4(pickleFile, get_top_n_kmers)